# Estimate rotational diffusion coefficients of Brownian particle

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools.quaternions as quats

In [ ]:
# Define particle class move and rotate function
class particle:
    def __init__(self, position, orientation):
        self.position = position
        self.orientation = orientation
        self.dim = len(self.position)
        
    def move(self,dt,D):
        dr = np.sqrt(2 * D * dt) * np.random.normal(0,1,self.dim)
        self.position += dr
        
    def rotate(self,dt,Drot):
        '''
        Donev implementation
        '''
        dphi = np.sqrt(2 * Drot* dt) * np.random.normal(0,1,self.dim)
        dquat = quats.angle2quat(dphi)
        self.orientation = quats.multiply(dquat,self.orientation)
        
    def rotate2(self,dt,Drot):
        '''
        Hummer implementation
        '''
        dphi = np.sqrt(Drot * dt/2) * np.random.normal(0,1,self.dim)
        dquat = np.zeros(4)
        dquat[1:] = dphi
        dquat[0] = np.sqrt(1 - 3 * Drot * dt/2)
        dquat = dquat/np.linalg.norm(dquat)
        self.orientation = quats.multiply(dquat,self.orientation)

In [ ]:
# Define simulation parameters
dt = 0.0001
Drot = 1.0
timesteps = 500000
x0 = np.array([0.0, 0.0, 0.0])
q0 = np.array([1.0, 0.0, 0.0, 0.0])

In [ ]:
# Euler Maruyama simulation
orientationArray = [q0]
particle1 = particle(x0,q0)
for i in range(timesteps):
    particle1.rotate(dt,Drot)
    orientationArray.append(1*particle1.orientation)

In [ ]:
# Calculate auto-correlation/mean-square displacement for several lagtimes
lagtimesIndexes = np.arange(100,2000,100)
lagtimes = np.zeros(len(lagtimesIndexes) + 1)
MSD_3D = np.zeros(len(lagtimesIndexes) + 1)
for i, lagtimeIndex in enumerate(lagtimesIndexes):
    lagtimes[i+1] = dt * lagtimeIndex
    MSD = 0.0
    for j in range(timesteps-lagtimeIndex):
        dq = orientationArray[j+lagtimeIndex] - orientationArray[j]
        MSD += dq*dq
    MSD = MSD/(timesteps - lagtimeIndex + 1)
    # MSD_3D = 3*(1-exp(-2Dtau))/4 ~ 3 * D * tau/2 (see Hummer paper appendix:  Anis Rot Diff)
    MSD_3D[i+1] = sum(MSD[1:])

In [ ]:
index = 15
DrotApprox = - np.log(1-4*MSD_3D[index]/3.0)/(2*lagtimes[index])
DrotApprox

In [ ]:
# Least square approximation with numpy
A = np.vstack([lagtimes, np.ones(len(lagtimes))]).T
y = - np.log(1-4*MSD_3D/3.0)/2
slope,b = np.linalg.lstsq(A, y, rcond=None)[0]
print(slope,b)

In [ ]:
plt.plot(lagtimes, - np.log(1-4*MSD_3D/3.0)/2, 'o', label='orientation')
plt.plot(lagtimes, slope*lagtimes + b, '-', label = 'orientation fit')
plt.legend()

In [ ]:
DrotApprox = slope
print(DrotApprox)

In [ ]:
# Least square approximation with numpy
A = np.vstack([lagtimes, np.ones(len(lagtimes))]).T
slope,b = np.linalg.lstsq(A, 2*MSD_3D/3, rcond=None)[0] # Approximation valid for small lagtimes
print(slope,b)